Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

In [5]:
# L2 regularization for logistic regression model.
# After testing a range of values, the hyperparameter 0.01 was chosen to roughly maximize validation set accuracy.

num_steps = 3001
batch_size = 128

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w_fc1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b_fc1 = tf.Variable(tf.zeros([num_labels]))
    tf_y = tf.matmul(tf_x, w_fc1) + b_fc1

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels)) + 0.01*tf.nn.l2_loss(w_fc1)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        if step % (train_labels.shape[0] // batch_size) == 0:
            p = np.random.permutation(train_labels.shape[0])
            train_dataset=train_dataset[p]
            train_labels=train_labels[p]

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions = session.run([optimizer, loss, tf_predictions], feed_dict = {tf_x:batch_data, tf_labels:batch_labels})

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:valid_dataset}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 44.960464
Minibatch accuracy: 11.7%
Validation accuracy: 15.1%
Minibatch loss at step 500: 0.851512
Minibatch accuracy: 82.8%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.926723
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.935863
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.756207
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.704028
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.668511
Minibatch accuracy: 85.2%
Validation accuracy: 81.0%
Test accuracy: 88.0%


In [6]:
# L2 regularization for neural network model.
# After testing a range of values, the hyperparameter 0.01 was chosen to roughly maximize validation set accuracy.

num_steps = 3001
batch_size = 128

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w_fc1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_fc1 = tf.nn.relu(tf.matmul(tf_x, w_fc1) + b_fc1)
    
    w_fc2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[num_labels]))
    tf_y = tf.matmul(h_fc1, w_fc2) + b_fc2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels)) + 0.01 * (tf.nn.l2_loss(w_fc1) + tf.nn.l2_loss(w_fc2))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        if step % (train_labels.shape[0] // batch_size) == 0:
            p = np.random.permutation(train_labels.shape[0])
            train_dataset=train_dataset[p]
            train_labels=train_labels[p]

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions = session.run([optimizer, loss, tf_predictions], feed_dict = {tf_x:batch_data, tf_labels:batch_labels})

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:valid_dataset}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 3459.675781
Minibatch accuracy: 11.7%
Validation accuracy: 33.5%
Minibatch loss at step 500: 21.211437
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.940220
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 0.788092
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.695876
Minibatch accuracy: 81.2%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.686249
Minibatch accuracy: 85.2%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.684145
Minibatch accuracy: 84.4%
Validation accuracy: 83.4%
Test accuracy: 90.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [7]:
# Restricting our training data to only ten batches causes overfitting.
# The minibatch accuracies quickly reach 100% as the model superficially learns all of the features in the training data.
# However, the low and stagnant validation accuracy indicates that the model has failed to learn properly.

num_steps = 501
batch_size = 128

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w_fc1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_fc1 = tf.nn.relu(tf.matmul(tf_x, w_fc1) + b_fc1)
    
    w_fc2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[num_labels]))
    tf_y = tf.matmul(h_fc1, w_fc2) + b_fc2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step % 10) * batch_size # restrict training to 10 batches
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions = session.run([optimizer, loss, tf_predictions], feed_dict = {tf_x:batch_data, tf_labels:batch_labels})

        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:valid_dataset}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 295.080505
Minibatch accuracy: 14.1%
Validation accuracy: 25.2%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Test accuracy: 85.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [8]:
# Dropout regularization reduces overfitting and improves test accuracy. 

num_steps = 501
batch_size = 128

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    
    keep_prob=tf.placeholder(tf.float32)
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w_fc1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    h_fc1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_x, w_fc1) + b_fc1), keep_prob)
    
    w_fc2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[num_labels]))
    tf_y = tf.matmul(h_fc1, w_fc2) + b_fc2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step % 10) * batch_size
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions = session.run([optimizer, loss, tf_predictions], feed_dict = {tf_x:batch_data, tf_labels:batch_labels, keep_prob: 0.5})

        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:valid_dataset, keep_prob: 1.0}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset, keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 480.923126
Minibatch accuracy: 8.6%
Validation accuracy: 26.1%
Minibatch loss at step 100: 11.529325
Minibatch accuracy: 94.5%
Validation accuracy: 78.1%
Minibatch loss at step 200: 2.366777
Minibatch accuracy: 97.7%
Validation accuracy: 79.1%
Minibatch loss at step 300: 3.419254
Minibatch accuracy: 97.7%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.171263
Minibatch accuracy: 98.4%
Validation accuracy: 79.2%
Minibatch loss at step 500: 3.323609
Minibatch accuracy: 95.3%
Validation accuracy: 79.0%
Test accuracy: 86.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [9]:
##### Neural network with two fully connected hidden layers, featuring 512 and 64 neurons.
# Using dropout + max norm regularization, linear learning rate decay, and AdamOptimizer.

num_steps = 15001
batch_size = 128
clip_norm = 1

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    global_step = tf.Variable(0)
    learning_rate = tf.train.polynomial_decay(1e-3, global_step, num_steps, 1e-4)
    
    keep_prob=tf.placeholder(tf.float32)
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w_fc1 = tf.Variable(tf.truncated_normal([image_size * image_size, 512]))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]))
    h_fc1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_x, w_fc1) + b_fc1), keep_prob)
    
    w_fc2 = tf.Variable(tf.truncated_normal([512, 64]))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_fc2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h_fc1, w_fc2) + b_fc2), keep_prob)

    w_fc3 = tf.Variable(tf.truncated_normal([64, num_labels]))
    b_fc3 = tf.Variable(tf.constant(0.1, shape=[num_labels]))
    tf_y = tf.matmul(h_fc2, w_fc3) + b_fc3

    with tf.control_dependencies([w_fc1.assign(tf.clip_by_norm(w_fc1, clip_norm, axes=1)), 
                                  w_fc2.assign(tf.clip_by_norm(w_fc2, clip_norm, axes=1)), 
                                  w_fc3.assign(tf.clip_by_norm(w_fc3, clip_norm, axes=1))]):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels))
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
        tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        if step % (train_labels.shape[0] // batch_size) == 0:
            p = np.random.permutation(train_labels.shape[0])
            train_dataset = train_dataset[p]
            train_labels = train_labels[p]
            
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions, lr = session.run([optimizer, loss, tf_predictions, learning_rate], feed_dict = {tf_x:batch_data, tf_labels:batch_labels, keep_prob: 0.8})

        if (step % 500 == 0):
            print("Learning rate at step %d: %f" % (step, lr))
            print("Minibatch loss: %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:valid_dataset, keep_prob: 1.0}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset, keep_prob: 1.0}), test_labels))

Initialized
Learning rate at step 0: 0.001000
Minibatch loss: 85.191391
Minibatch accuracy: 4.7%
Validation accuracy: 26.4%
Learning rate at step 500: 0.000970
Minibatch loss: 0.575379
Minibatch accuracy: 81.2%
Validation accuracy: 83.2%
Learning rate at step 1000: 0.000940
Minibatch loss: 0.604766
Minibatch accuracy: 79.7%
Validation accuracy: 85.0%
Learning rate at step 1500: 0.000910
Minibatch loss: 0.389900
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Learning rate at step 2000: 0.000880
Minibatch loss: 0.623073
Minibatch accuracy: 80.5%
Validation accuracy: 85.8%
Learning rate at step 2500: 0.000850
Minibatch loss: 0.492281
Minibatch accuracy: 86.7%
Validation accuracy: 86.4%
Learning rate at step 3000: 0.000820
Minibatch loss: 0.372807
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
Learning rate at step 3500: 0.000790
Minibatch loss: 0.605494
Minibatch accuracy: 82.8%
Validation accuracy: 87.0%
Learning rate at step 4000: 0.000760
Minibatch loss: 0.384651
Minibatch 